In [1]:
!pip install langchain faiss-cpu sentence-transformers PyPDF2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install pypdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U langchain-community

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade langchain-ollama

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade langchain-huggingface

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Imports
from pathlib import Path
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import Ollama


In [7]:
# Config
from langchain_huggingface import HuggingFaceEmbeddings

PDF_FOLDER = r"C:\Users\visarati\Desktop\Notebooks\pdfs"    # <-- folder path, not file  # folder where PDFs are stored
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
OLLAMA_MODEL = "llama2"

In [8]:
# Load and split PDF
docs = []
for pdf in Path(PDF_FOLDER).glob("*.pdf"):
    loader = PyPDFLoader(str(pdf))
    docs.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

print(f"Loaded {len(docs)} pages → split into {len(chunks)} chunks")


Loaded 13 pages → split into 40 chunks


In [9]:
# Create FAISS index
embedder = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
vectorstore = FAISS.from_documents(chunks, embedder)

In [10]:
# Setup Ollama LLM
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model=OLLAMA_MODEL)

In [11]:
# Ask a question
def answer_question(question, k=3, show_chunks=True):
    # Retrieve top-k chunks
    docs = vectorstore.similarity_search(question, k=k)
    context = "\n\n".join([d.page_content for d in docs])

    if show_chunks:
        print("Retrieved chunks:\n")
        for i, d in enumerate(docs):
            print(f"--- Chunk {i+1} ---")
            print(d.page_content[:300], "...\n")  # show first 300 chars
    
    # Build prompt
    prompt = f"Answer the question based only on the context.\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"
    
    return llm(prompt)


In [13]:
# Try it out
answer = answer_question.invoke("What is the theme of the document and how is the company using generative AI?")
print("\n--- ANSWER ---\n")
print(answer)

AttributeError: 'function' object has no attribute 'invoke'